In [ ]:
# Start writing code here...

## Bayesian Estimation Supersedes the T-test

## The Problem
Several statistical inference procedures involve the comparison of two groups. We may be interested in whether one group is larger than another, or simply different from the other. We require a statistical model for this because true differences are usually accompanied by measurement or stochastic noise that prevent us from drawing conclusions simply from differences calculated from the observed data.

The de facto standard for statistically comparing two (or more) samples is to use a statistical test. This involves expressing a null hypothesis, which typically claims that there is no difference between the groups, and using a chosen test statistic to determine whether the distribution of the observed data is plausible under the hypothesis. This rejection occurs when the calculated test statistic is higher than some pre-specified threshold value.

Unfortunately, it is not easy to conduct hypothesis tests correctly, and their results are very easy to misinterpret. Setting up a statistical test involves several subjective choices (e.g. statistical test to use, null hypothesis to test, significance level) by the user that are rarely justified based on the problem or decision at hand, but rather, are usually based on traditional choices that are entirely arbitrary (Johnson 1999). The evidence that it provides to the user is indirect, incomplete, and typically overstates the evidence against the null hypothesis (Goodman 1999).

A more informative and effective approach for comparing groups is one based on estimation rather than testing, and is driven by Bayesian probability rather than frequentist. That is, rather than testing whether two groups are different, we instead pursue an estimate of how different they are, which is fundamentally more informative. Moreover, we include an estimate of uncertainty associated with that difference which includes uncertainty due to our lack of knowledge of the model parameters (epistemic uncertainty) and uncertainty due to the inherent stochasticity of the system (aleatory uncertainty).

To illustrate how this Bayesian estimation approach works in practice, we will use a fictitious example from Kruschke (2012) concerning the evaluation of a clinical trial for drug evaluation. The trial aims to evaluate the efficacy of a “smart drug” that is supposed to increase intelligence by comparing IQ scores of individuals in a treatment arm (those receiving the drug) to those in a control arm (those recieving a placebo). There are 47 individuals and 42 individuals in the treatment and control arms, respectively.

## Toy Example

In [ ]:
drug = (101,100,102,104,102,97,105,105,98,101,100,123,105,103,100,95,102,106,
        109,102,82,102,100,102,102,101,102,102,103,103,97,97,103,101,97,104,
        96,103,124,101,101,100,101,101,104,100,101)
placebo = (99,101,100,101,102,100,97,101,104,101,102,102,100,105,88,101,100,
           104,100,100,100,101,102,103,97,101,101,100,101,99,101,100,100,
           101,100,99,101,100,102,99,100,99)

y1 = np.array(drug)
y2 = np.array(placebo)

# What does np.r_ do (numpy)?
# What it does is row-wise merging. https://stackoverflow.com/questions/30597869/what-does-np-r-do-numpy
y = pd.DataFrame(dict(value = np.r_[y1, y2], 
                      group = np.r_[["drug"] * len(drug), ["placebo"] * len(placebo)]))



In [ ]:
# Prior for group mean
mu_m = y.value.mean()
mu_s = y.value.std() * 2

# Prior for group standard deviation
sigma_low = 1
sigma_high = 10

# Set up prior for 2 groups
with pm.Model() as model:
    # Normal prior for group mean k = 1,2
    group1_mean = pm.Normal('group1_mean', mu = mu_m, sd = mu_s)
    group2_mean = pm.Normal('group2_mean', mu = mu_m, sd = mu_s)
    
    # Uniform prior for group standard deviation
    group1_std = pm.Uniform('group1_std', lower = sigma_low, upper = sigma_high)
    group2_std = pm.Uniform('group2_std', lower = sigma_low, upper = sigma_high)
    
    # Exponential prior for degree of freedom
    v = pm.Exponential('v_minus_one', 1/29) + 1
    
    # Transform the standard deviation to precision
    lambda_1 = group1_std ** -2
    lambda_2 = group2_std ** -2
    
    # Assign likelihood using T-distribution
    # You can see that we use the same prior for both drug and placebo population
    group1 = pm.StudentT("drug", nu = v, mu = group1_mean, lam = lambda_1, observed = y1)
    group2 = pm.StudentT("placebo", nu = v, mu = group2_mean, lam = lambda_2, observed = y2)
    
    # Deterministic nodes for difference of means and standard deviations between two groups
    diff_means = pm.Deterministic("difference of means", group1_mean - group2_mean)
    diff_stds = pm.Deterministic("difference of stds", group1_std - group2_std)
    effect_size = pm.Deterministic("effect size", diff_means / np.sqrt((group1_std ** 2 + group2_std **2) / 2))
    
    # Fit the model and evaluate the output as trace
    trace = pm.sample(2000, chains = 4, tune = 500)

In [ ]:
# The first step of visualization comes with using the plot_posterior function. 
# It takes the trace object and show the posterior mean, standard deviation and degree of freedom of the model
pm.plot_posterior(
    trace,
    var_names=["group1_mean", "group2_mean", "group1_std", "group2_std", "v_minus_one"],
    color="#87ceeb",
)

Looking at the group differences below, we can conclude that there are meaningful differences between the two groups for all three measures. For these comparisons, it is useful to use zero as a reference value (ref_val); providing this reference value yields cumulative probabilities for the posterior distribution on either side of the value. Thus, for the difference of means, at least 97% of the posterior probability are greater than zero, which suggests the group means are credibly different. The effect size and differences in standard deviation are similarly positive.

In [ ]:
# We move on to plot the posterior distribution of the difference in means, standard deviation and effect size of the two groups
pm.plot_posterior(trace,
                  var_names = ["difference of means", "difference of stds", "effect size"],
                  ref_val = 0,
                  color = "#87ceeb")

When forestplot is called on a trace with more than one chain, it also plots the potential scale reduction parameter, which is used to reveal evidence for lack of convergence; values near one, as we have here, suggest that the model has converged.

In [ ]:
# Now we make a forest plot to see how much mean does the two groups differ
pm.forestplot(trace, 
          var_names = ["group1_mean", "group2_mean"])

In [ ]:
# We also want to see how much spread difference occurred between the two groups by comparing standard deviation
pm.forestplot(trace, 
          var_names = ["group1_std", "group2_std"])

In [ ]:
# Finally we look at the parameters in table-like view using pm.summary
pm.summary(trace, varnames = ["difference of means", "difference of stds", "effect size"])